In [502]:
import re
import time
import pandas as pd
import json
import requests
import FinanceDataReader as fdr
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime

In [412]:
#data 불러오기
stock = pd.read_csv('./recommend_stock.csv')

In [413]:
#date 타입 날짜 문자로 바꾸기
stock['end_date'] = stock.date.apply(lambda x : re.sub("-","",x))
stock['end_date']= stock['end_date'].astype('str')

#date date타입으로 변환
stock['date'] = pd.to_datetime(stock['date'])

In [415]:
stocks = stock

In [416]:
stocks

,date,stock_name,stock_code,end_date
0,2019-10-29,알리코제약,260660,20191029
1,2019-10-29,KPX생명과학,114450,20191029
2,2019-10-29,제일바이오,052670,20191029
3,2019-10-30,아스타,246720,20191030
4,2019-10-30,SM,063440,20191030
...,...,...,...,...
977,2023-02-01,오픈베이스,049480,20230201
978,2023-02-01,일성건설,013360,20230201
979,2023-02-02,선익시스템,171090,20230202
980,2023-02-02,누리플렉스,040160,20230202


In [5]:
#한국투자 API 보안 인증키 발급 
APP_KEY = "PSDbkNNrIX3j60cW2MEFnBGZDZ1BzpgDxvC0"
APP_SECRET = "QfaZRX24K15ojO4yjG5o5dM+SUvJT4PYiK+tKKzgikyhmfrnTXoxOwn9rCf2hzXdtOhh4v3/O4bSFfQgS+u6ojdb6KgWuyub4GRraoWYaJTJyHSRqWUPab2GiCvrI/aIjx5S925MohrETq0k/eopFHgMInG0lmZdAh5TaegaZ1NDpNvPW8Y="
URL_BASE = "https://openapi.koreainvestment.com:9443"
PATH = "oauth2/tokenP"
URL = f"{URL_BASE}/{PATH}"

headers = {"content-type":"application/json"}
body = {"grant_type":"client_credentials",
        "appkey":APP_KEY, 
        "appsecret":APP_SECRET}

res = requests.post(URL, headers=headers, data=json.dumps(body))
ACCESS_TOKEN = res.json()["access_token"]

In [417]:
#일자별 시세 가져오기 (함수)
def get_info_from_kisapi(stock_code,end_date):
    time.sleep(0.1)
    
    st_date = pd.date_range(end = end_date,periods=20)[0]
    st_date = st_date.strftime("%Y%m%d")
    
    URL = 'https://openapi.koreainvestment.com:9443/uapi/domestic-stock/v1/quotations/inquire-daily-itemchartprice'

    headers = {"Content-Type":"application/json", 
            "authorization": f"Bearer {ACCESS_TOKEN}",
            "appKey":APP_KEY,
            "appSecret":APP_SECRET,
            "tr_id":"FHKST03010100"
            }

    params = {
        "FID_COND_MRKT_DIV_CODE":"J",
        "FID_INPUT_ISCD":stock_code,
        "FID_INPUT_DATE_1" : st_date,
        "FID_INPUT_DATE_2" : end_date,
        "FID_PERIOD_DIV_CODE" : "D",
        "FID_ORG_ADJ_PRC" : "1"
    }

    res = requests.get(URL, headers=headers, params=params)   
    res = res.json().get('output2')[0:11]
    res.reverse()
    return res

In [369]:
#종목데이터 추출
temp = pd.DataFrame()
day = 10
for i in res:

    data = pd.DataFrame({
        'stck_oprc' :  i.get('stck_oprc'),
        'stck_clpr' : i.get('stck_clpr'),
        'stck_hgpr' : i.get('stck_hgpr'),
        'stck_lwpr' : i.get('stck_lwpr'),
        'acml_vol'  : i.get('acml_vol')},
        index=[f'D-{day}']
    )
    temp = pd.concat([temp,data])
    day -=1
temp= temp.transpose()

In [360]:
#세부데이터 추출 편집
#전일종가
br_pr = temp.iloc[1:2,9:10].reset_index(drop = True)
br_pr.columns = 'br_pr_'+ br_pr.columns
#당일시가
tr_pr = temp.iloc[2:3,10:].reset_index(drop = True)
tr_pr.columns = 'tr_pr_'+ tr_pr.columns
#거래량(10일)
br_vol = temp.iloc[4:5,0:10].reset_index(drop = True)
br_vol.columns = 'Volume_'+ br_vol.columns
#저점(10일)
br_lpr = temp.iloc[3:4,0:10].reset_index(drop = True)
br_lpr.columns = 'lpr_'+ br_lpr.columns
#데이터 합치기 
data = pd.concat([br_pr,tr_pr,br_vol,br_lpr], axis=1)
#인덱스 삽입
date_data = pd.DataFrame({'date' : end_date,
                          'stock_code' : stock_code}
                          ,index=[0])
data = pd.concat([date_data,data], axis =1 )


In [418]:
#종목 일자별 데이터 가져오기 (함수)
def get_info_from_stock(res):
    temp = pd.DataFrame()
    day = 10
    for i in res:

        data = pd.DataFrame({
            'stck_oprc' :  i.get('stck_oprc'),
            'stck_clpr' : i.get('stck_clpr'),
            'stck_hgpr' : i.get('stck_hgpr'),
            'stck_lwpr' : i.get('stck_lwpr'),
            'acml_vol'  : i.get('acml_vol')},
            index=[f'D-{day}']
        )
        temp = pd.concat([temp,data])
        day -=1
    temp= temp.transpose()
    
    #전일종가
    br_pr = temp.iloc[1:2,9:10].reset_index(drop = True)
    br_pr.columns = 'br_pr_'+ br_pr.columns
    #당일시가
    tr_pr = temp.iloc[2:3,10:].reset_index(drop = True)
    tr_pr.columns = 'tr_pr_'+ tr_pr.columns
    #거래량(10일)
    br_vol = temp.iloc[4:5,0:10].reset_index(drop = True)
    br_vol.columns = 'Volume_'+ br_vol.columns
    #저점(10일)
    br_lpr = temp.iloc[3:4,0:10].reset_index(drop = True)
    br_lpr.columns = 'lpr_'+ br_lpr.columns
    #데이터 합치기 
    data = pd.concat([br_pr,tr_pr,br_vol,br_lpr], axis=1)
    #인덱스 삽입
    date_data = pd.DataFrame({'date' : end_date,
                            'stock_code' : stock_code}
                            ,index=[0])
    data = pd.concat([date_data,data], axis =1 )
    
    return data   

In [433]:
#전종목 가져오기
data = pd.DataFrame()
for i in range(len(stocks)):
    end_date = stocks.end_date[i]
    stock_code = stocks.stock_code[i]
    res = get_info_from_kisapi(stock_code,date)
    temp = get_info_from_stock(res)
    data = pd.concat([data,temp],axis=0)
data = data.drop(columns=['date','stock_code'])
data = data.reset_index().drop(columns = 'index')

In [450]:
stock2 = pd.concat([stocks, data], axis=1)

In [452]:
stock2.to_csv('stock2.csv', index=False, encoding='utf-8-sig')

In [463]:
stock2

,date,stock_name,stock_code,end_date,br_pr_D-1,tr_pr_D-0,Volume_D-10,Volume_D-9,Volume_D-8,Volume_D-7,...,lpr_D-10,lpr_D-9,lpr_D-8,lpr_D-7,lpr_D-6,lpr_D-5,lpr_D-4,lpr_D-3,lpr_D-2,lpr_D-1
0,2019-10-29,알리코제약,260660,20191029,6710,6700,12323,13702,35438,24032,...,6350,6350,6420,6460,6500,6490,6420,6450,6440,6590
1,2019-10-29,KPX생명과학,114450,20191029,4785,4785,213900,473545,524945,453247,...,5070,5000,5070,5010,4990,4780,4705,4735,4750,4780
2,2019-10-29,제일바이오,052670,20191029,1307,1320,14358,24826,31035,25647,...,1230,1230,1240,1250,1251,1251,1245,1243,1254,1298
3,2019-10-30,아스타,246720,20191030,4140,4180,11841,14872,24155,25936,...,3670,3660,3675,3765,3670,3780,3885,4050,4080,4090
4,2019-10-30,SM,063440,20191030,2570,2690,1463721,3778711,1098590,465803,...,2425,2470,2480,2480,2450,2475,2435,2515,2500,2550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977,2023-02-01,오픈베이스,049480,20230201,3145,3130,1161373,476161,9668849,6418084,...,2795,2795,2885,3030,3075,3090,3115,3195,3185,3145
978,2023-02-01,일성건설,013360,20230201,2690,2715,157697,471438,196895,200226,...,2325,2315,2380,2430,2490,2540,2580,2620,2675,2685
979,2023-02-02,선익시스템,171090,20230202,22650,23300,306938,279798,560907,409859,...,19150,19350,19580,20100,20200,20300,19930,20000,21450,21900
980,2023-02-02,누리플렉스,040160,20230202,6170,6270,519941,215682,197763,378487,...,5580,5660,5560,5660,5760,5710,5710,5760,6020,6150


In [474]:
stock2.columns[4:]
for cn in stock2.columns[4:]:
    stock2[f'{cn}'] = stock2[f'{cn}'].apply(lambda x : int(x))

In [483]:
stock2['gap_change'] = stock2.iloc[:,4:6].pct_change(axis =1).dropna(axis=1)

In [495]:
stock2['mean_vlome'] = stock2.iloc[:,6:16].mean(axis =1)

In [541]:
parmeter = stock2.iloc[:,16:26]

In [543]:
parmeter = parmeter.iloc[0:1,:]

In [544]:
parmeter

,lpr_D-10,lpr_D-9,lpr_D-8,lpr_D-7,lpr_D-6,lpr_D-5,lpr_D-4,lpr_D-3,lpr_D-2,lpr_D-1
0,6350,6350,6420,6460,6500,6490,6420,6450,6440,6590


In [545]:
parmeter = parmeter.T.reset_index()

In [549]:
parmeter = parmeter.rename(columns={0:'price'})


In [556]:
parmeter = parmeter.drop(columns = 'index').reset_index()

In [560]:
parmeter['index'] = 10-parmeter['index']

In [561]:
model = sm.OLS.from_formula('index ~ price', parmeter).fit()

In [562]:
model.params

Intercept    204.097566
price         -0.030805
dtype: float64